### Implementing a keras model in Tensoflow
Author : @leopauly | cnlp@leeds.ac.uk

In [1]:
##Imports
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Input
from keras.models import Model
import numpy as np
import pylab as plt
import os
from keras import backend as K

K.set_learning_phase(1) #set learning phase

Using TensorFlow backend.


In [2]:
## Defining variables
batch_size=512
nb_classes=10
lr_rate=.0005
height=28
width=28
channel=1
input_shape=[height,width,channel]
iterations=500

#### Loading MNIST data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Defining model

In [4]:
## Defining placeholders in tf
data = tf.placeholder(tf.float32, [None, height,width,channel]) #step_size=No: of frames in video sequence
target = tf.placeholder(tf.float32, [None, nb_classes])

In [5]:
## Defining the model in keras using functional layers
input_layer = Input(shape=input_shape)
layer1= Conv2D(32, kernel_size=(3, 3),activation='relu')(input_layer)
layer2=Conv2D(64, (3, 3), activation='relu')(layer1)
layer3=MaxPooling2D(pool_size=(2, 2))(layer2)
layer4=Flatten()(layer3)
layer5=Dense(128, activation='relu')(layer4)
layer6=Dropout(0.5)(layer5)
output_layer=Dense(nb_classes, activation='linear')(layer6)
model=Model(input_layer,output_layer)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
__________

In [6]:
# Defining optimisation method and calc
out=model(data)
predictions= tf.nn.softmax(out)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target,logits=out))
optimizer = tf.train.AdamOptimizer(lr_rate)
minimize = optimizer.minimize(cross_entropy)
mistakes = tf.equal(tf.argmax(target, 1), tf.argmax(predictions, 1))
accuracy = tf.reduce_mean(tf.cast(mistakes, tf.float32))

# Defining variables for writing summary
tf.summary.histogram("predictions",predictions )
tf.summary.scalar("cross_entropy",cross_entropy )
summary_op = tf.summary.merge_all()

#### Training model

In [7]:
# The keras model is trained as a tensorlfow graph
init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)

for i in range(iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape(batch_x.shape[0], height, width, channel)
    sess.run([minimize],{data: batch_x, target: batch_y})
    print('Iteration {}'.format(i))

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [8]:
mnist_test = mnist.test.images.reshape(mnist.test.images.shape[0], height, width, channel)
acc=sess.run(accuracy, feed_dict={data:mnist_test ,target: mnist.test.labels})
print('accuracy on test set: {}'.format(acc))

accuracy on test set: 0.9791001081466675


#### Clossing session

In [9]:
sess.close()